# Análisis de Datos de Google Sheets

Este notebook permite cargar, visualizar y analizar datos nutricionales de jugadores de rugby desde Google Sheets.

In [3]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import plotly.express as px
import plotly.graph_objects as go
from PIL import Image
import os

# Configurar estilo de visualización
plt.style.use('ggplot')
sns.set_style('whitegrid')
%matplotlib inline

## Carga de Datos desde Google Sheets

La siguiente función permite cargar datos directamente desde una hoja de Google Sheets compartida.

In [7]:
def cargar_datos_google_sheets(sheet_url=None):
    if sheet_url is None:
        # URL de la hoja de cálculo compartida por defecto
        sheet_url = "https://docs.google.com/spreadsheets/d/17rtbwMwGMXvoE4sXdGNXXr19z73mdCSjxTApoG5v_6o/edit?gid=0#gid=0"
    
    # Extraer el ID de la hoja de cálculo de la URL
    try:
        if 'spreadsheets/d/' in sheet_url:
            sheet_id = sheet_url.split('spreadsheets/d/')[1].split('/')[0]
        else:
            # Si no tiene el formato esperado
            raise ValueError("La URL no parece ser una URL válida de Google Sheets")
        
        # Construir la URL para exportar como CSV
        csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"
        
        # Cargar los datos desde la URL
        print(f"📊 Cargando datos desde Google Sheets...")
        df = pd.read_csv(csv_url)
        
        # Mostrar información sobre los datos cargados
        print(f"✅ Datos cargados exitosamente: {df.shape[0]} filas y {df.shape[1]} columnas")
        
        # Procesar las columnas de fecha si existen
        if 'Fecha' in df.columns:
            try:
                df['fecha'] = pd.to_datetime(df['Fecha'], format='%d/%m/%Y')
                df['fecha_str'] = df['fecha'].dt.strftime('%d/%m/%Y')
                print("✅ Columna de fecha procesada correctamente")
            except Exception as e:
                print(f"⚠️ No se pudo procesar la columna de fecha: {e}")
        
        return df
    
    except Exception as e:
        print(f"❌ Error al cargar los datos: {e}")
        return pd.DataFrame()  # Devolver un DataFrame vacío en caso de error
      


In [9]:
def normalizar_valores_nan(dataframe):
    """
    Normaliza el DataFrame reemplazando los valores NaN por 0.
    
    Args:
        dataframe (pd.DataFrame): El DataFrame a normalizar
        
    Returns:
        pd.DataFrame: DataFrame con valores NaN reemplazados por 0
    """
    try:
        # Crear una copia para no modificar el original directamente
        df_normalizado = dataframe.copy()
        
        # Reemplazar todos los valores NaN por 0
        df_normalizado = df_normalizado.fillna(0)
        
        # Contar cuántos valores fueron reemplazados
        total_nan = dataframe.isna().sum().sum()
        print(f"✅ Se han normalizado {total_nan} valores NaN a 0 en el DataFrame")
        
        # Mostrar un resumen de las columnas afectadas
        columnas_afectadas = dataframe.columns[dataframe.isna().any()].tolist()
        if columnas_afectadas:
            print("\nColumnas normalizadas:")
            for col in columnas_afectadas:
                num_nan = dataframe[col].isna().sum()
                print(f"- {col}: {num_nan} valores reemplazados")
                
        return df_normalizado
    except Exception as e:
        print(f"❌ Error al normalizar valores: {e}")
        return dataframe  # Devolver el DataFrame original en caso de error


## Carga de Datos

Puedes usar la URL por defecto o introducir tu propia URL de Google Sheets.

## Información General del DataFrame

Veamos información básica sobre los datos cargados.

In [11]:
df.columns


Index(['fecha', 'Apellido', 'DNI', 'Categoria', 'EMAIL', 'Posición ',
       'Peso (kg)', 'Talla (cm)', 'IMC', 'Talla sentado (cm)', 'kg MA',
       '% MA ', 'Z Adiposo', '6 Pliegues', 'kg MM', '% MM ', 'Z MM',
       'kg de MO', 'IMO', 'Objetivo 1'],
      dtype='object')

In [12]:
def mostrar_info_jugador(apellido):
    """
    Muestra un dashboard con información personal del jugador seleccionado.
    
    Args:
        apellido (str): Apellido del jugador a buscar
    """
    try:
        # Buscar el jugador en el DataFrame (ignorando mayúsculas/minúsculas)
        jugador = df[df['Apellido'].str.contains(apellido, case=False, na=False)]
        
        if len(jugador) == 0:
            print(f"❌ No se encontró ningún jugador con el apellido '{apellido}'")
            return
        
        # Tomar el primer registro si hay múltiples coincidencias
        jugador = jugador.iloc[0]
        
        # Crear el dashboard usando caracteres ASCII
        print("\n" + "="*50)
        print("🏉 DASHBOARD DE COMPOSICIÓN CORPORAL - JUGADOR DE RUGBY")
        print("="*50)
        
        print("\n🧍‍♂️ INFORMACIÓN PERSONAL")
        print("-"*30)
        print(f"👤 Nombre: {jugador['Apellido']}")
        
        # Verificar si existe cada columna antes de mostrarla
        if 'Posición' in jugador:
            print(f"🏈 Posición: {jugador['Posición']}")
        elif 'Posicion' in jugador:  # Alternativa sin tilde
            print(f"🏈 Posición: {jugador['Posicion']}")
        
        if 'Talla (cm)' in jugador:
            print(f"📏 Altura: {jugador['Talla (cm)']} cm")
        elif 'Altura' in jugador:
            print(f"📏 Altura: {jugador['Altura']} cm")
        
        if 'Peso (kg)' in jugador:
            print(f"⚖️ Peso: {jugador['Peso (kg)']} kg")
        elif 'Peso' in jugador:
            print(f"⚖️ Peso: {jugador['Peso']} kg")
        
        if 'fecha_str' in jugador:
            print(f"📅 Última medición: {jugador['fecha_str']}")
        
        if 'Objetivo 1' in jugador:
            print(f"🎯 Objetivo: {jugador['Objetivo 1']}")
        elif 'Objetivo' in jugador:
            print(f"🎯 Objetivo: {jugador['Objetivo']}")
        
        print("\n" + "="*50)
        
    except Exception as e:
        print(f"❌ Error al procesar la información: {e}")
        
        # Mostrar las columnas disponibles para ayudar a depurar
        print("\nColumnas disponibles en el DataFrame:")
        for i, col in enumerate(df.columns):
            print(f"{i+1}. {col}")
        
    return

In [13]:
d = mostrar_info_jugador("Alvarez, Dylan")  # Cambia "Gonzalez" por el apellido que desees buscar


🏉 DASHBOARD DE COMPOSICIÓN CORPORAL - JUGADOR DE RUGBY

🧍‍♂️ INFORMACIÓN PERSONAL
------------------------------
👤 Nombre: Alvarez, Dylan
📏 Altura: 177,00 cm
⚖️ Peso: 82,1 kg
🎯 Objetivo: Aumento de Masa Muscular



In [15]:
df

,fecha,Apellido,DNI,Categoria,EMAIL,Posición,Peso (kg),Talla (cm),IMC,Talla sentado (cm),kg MA,% MA,Z Adiposo,6 Pliegues,kg MM,% MM,Z MM,kg de MO,IMO,Objetivo 1
0,3-2-2025,"Alegre, Diego",30281398,0,0.0,Pilar,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3-2-2025,"Alvarez, Dylan",45742224,0,0.0,Wing,"82,1","177,00","26,20",92,"18,7","22,78","-1,38",71,"41,58","50,65","2,67","8,62","4,82",Aumento de Masa Muscular
2,3-2-2025,"Amigorena, Gastón",26106150,0,0.0,Pilar,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3-2-2025,"Amigorena, Valentin",46892621,0,0.0,Wing,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,11-03-2025,"Arévalo, Michay Federico",45742426,0,0.0,Medio Scrum,"64,7","164,00","24,05",90,"14,1","21,8","-1,62",58,"32,6","50,4","2,36","6,5","4,9",Aumento de Masa Muscular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,0,0,#REF!,Plantel Superior,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
185,0,0,#REF!,Plantel Superior,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
186,0,0,#REF!,Plantel Superior,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
187,0,0,#REF!,Plantel Superior,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
from IPython.display import display, clear_output
import ipywidgets as widgets

def seleccionar_jugador(dataframe):
    """
    Permite seleccionar interactivamente un jugador del DataFrame y muestra su información.
    
    Args:
        dataframe (pd.DataFrame): DataFrame con los datos de los jugadores
    
    Returns:
        dict: Datos del jugador seleccionado
    """
    try:
        # Verificar que existe la columna 'Apellido'
        if 'Apellido' not in dataframe.columns:
            print("❌ Error: No se encontró la columna 'Apellido' en el DataFrame")
            return None
        
        # Obtener la lista de jugadores únicos y ordenarlos alfabéticamente
        jugadores = sorted(dataframe['Apellido'].unique())
        
        if len(jugadores) == 0:
            print("❌ No hay jugadores en el DataFrame")
            return None
        
        # Crear un dropdown para seleccionar jugadores
        dropdown = widgets.Dropdown(
            options=jugadores,
            description='Jugador:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='50%')
        )
        
        output = widgets.Output()
        
        # Función que se ejecuta cuando se selecciona un jugador
        def on_jugador_change(change):
            with output:
                clear_output()
                if change['new']:
                    # Mostrar información del jugador seleccionado
                    mostrar_info_jugador(change['new'])
                    
                    # Devolver los datos del jugador seleccionado
                    return dataframe[dataframe['Apellido'] == change['new']].iloc[0].to_dict()
        
        # Conectar el evento de cambio con la función
        dropdown.observe(on_jugador_change, names='value')
        
        # Mostrar el dropdown y el output
        print("🏉 Selecciona un jugador para ver su información:")
        display(dropdown)
        display(output)
        
        # Mostrar el primer jugador por defecto
        if jugadores:
            with output:
                mostrar_info_jugador(jugadores[0])
        
    except Exception as e:
        print(f"❌ Error al crear el selector de jugadores: {e}")
        return None


